In [1]:
import numpy as np
import pandas as pd
import redis
import os

In [2]:
# Connect to Redis
hostname=os.getenv('REDIS_HOST')
port=os.getenv('REDIS_PORT')
password=os.getenv('REDIS_PASSWORD')

r = redis.Redis(host=hostname, port=port, password=password)

In [3]:
r.ping()

True

In [4]:
# load the data (nzip file)
np_file = np.load('datasets/face_recognition.npz', allow_pickle=True)
np_file.files

['xvalues', 'col_names']

In [5]:
x_values = np_file['xvalues']
col_names = np_file['col_names']

df = pd.DataFrame(x_values, columns=col_names)
df

,name,role,facial_features
0,Angelina Jolie,Student,"[0.43242034, 1.0748498, -0.12862365, -1.833291..."
1,Barack Obama,Teacher,"[0.94858104, -0.5584052, 0.094820715, -1.27965..."
2,Chris Evans,Student,"[0.7201715, -1.4684149, -0.7781253, -1.4601073..."
3,Morgan Freeman,Teacher,"[0.48535103, -0.6192555, -0.4069932, 1.0687788..."
4,Scarlett Johansson,Student,"[0.19131963, -0.48843804, -1.690233, 0.4423967..."


In [6]:
df['role_name'] = df['role']+'@'+df['name']
df

,name,role,facial_features,role_name
0,Angelina Jolie,Student,"[0.43242034, 1.0748498, -0.12862365, -1.833291...",Student@Angelina Jolie
1,Barack Obama,Teacher,"[0.94858104, -0.5584052, 0.094820715, -1.27965...",Teacher@Barack Obama
2,Chris Evans,Student,"[0.7201715, -1.4684149, -0.7781253, -1.4601073...",Student@Chris Evans
3,Morgan Freeman,Teacher,"[0.48535103, -0.6192555, -0.4069932, 1.0687788...",Teacher@Morgan Freeman
4,Scarlett Johansson,Student,"[0.19131963, -0.48843804, -1.690233, 0.4423967...",Student@Scarlett Johansson


In [7]:
records = df[['role_name', 'facial_features']].to_dict('records')

In [8]:
len(records)

5

### Save data in Redis

In [9]:
for record in records:
    role_name = record['role_name']
    vector = record['facial_features']

    # convert numpy array to bytes
    vector_bytes = vector.tobytes()

    # save to redis
    r.hset(name='academy:register', key=role_name, value=vector_bytes)

### Retrive Records from Redis

In [10]:
retrieve_dict = r.hgetall('academy:register')
retrieve_series = pd.Series(retrieve_dict)
retrieve_series

b'Student@Chris Evans'           b')]8?\x05\xf5\xbb\xbf83G\xbf\xcc\xe4\xba\xbf\...
b'Student@Scarlett Johansson'    b'K\xe9C>\x8d\x14\xfa\xbe\x8eY\xd8\xbf\xd3\x81...
b'Teacher@Barack Obama'          b'5\xd6r?\xa5\xf3\x0e\xbf]1\xc2=\xde\xcb\xa3\x...
b'Teacher@Morgan Freeman'        b'\xee\x7f\xf8>\x87\x87\x1e\xbfja\xd0\xbe\xbe\...
b'Student@Angelina Jolie'        b'3f\xdd>\xae\x94\x89?\xeb\xb5\x03\xbeN\xa9\xe...
dtype: object

In [11]:
retrieve_series = retrieve_series.apply(lambda x: np.frombuffer(x, dtype=np.float32))
index = retrieve_series.index
index = list(map(lambda x: x.decode('utf-8'), index))

In [12]:
retrieve_series.index = index

In [13]:
retrieve_df = retrieve_series.to_frame().reset_index()
retrieve_df.columns = ['role_name', 'facial_features']
retrieve_df

,role_name,facial_features
0,Student@Chris Evans,"[0.7201715, -1.4684149, -0.7781253, -1.4601073..."
1,Student@Scarlett Johansson,"[0.19131963, -0.48843804, -1.690233, 0.4423967..."
2,Teacher@Barack Obama,"[0.94858104, -0.5584052, 0.094820715, -1.27965..."
3,Teacher@Morgan Freeman,"[0.48535103, -0.6192555, -0.4069932, 1.0687788..."
4,Student@Angelina Jolie,"[0.43242034, 1.0748498, -0.12862365, -1.833291..."


## Registration Form

In [14]:
import cv2
from insightface.app import FaceAnalysis

In [15]:
# configure face analysis
app = FaceAnalysis(name='buffalo_sc', root='', providers=['CUDAExecutionProvider'])

app.prepare(ctx_id=0, det_size=(640, 640), det_thresh=0.5)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: models\buffalo_sc\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: models\buffalo_sc\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


#### Collect person information

In [16]:
# python input
name = input('Enter your name: ')
role = input("""
Please select your role:
1. Student
2. Teacher
             
Enter your role:
 """)


if role in ('1', '2'):
    if role == '1':
        role = 'Student'
    else:
        role = 'Teacher'
else:
    print('Invalid role. Please try again.')
        
key = role+'@'+name
print('Your name is: ', name)
print('Your role is: ', role)
print('Your key is: ', key)


Your name is:  Awaliyatul Hikmah
Your role is:  Student
Your key is:  Student@Awaliyatul Hikmah


#### Collect the sample using video capture

In [17]:
cap = cv2.VideoCapture(0) # 0: default camera, 1: external camera
face_embeddings = []
sample = 0
while True:
    ret, frame = cap.read()
    
    if not ret:
        print('Error: Unable to capture video')
        break

    # get result from face analysis
    results = app.get(frame, max_num=1) # max_num: maximum number of faces to detect
    
    for res in results:
        sample += 1
        x1, y1, x2, y2 = res['bbox'].astype(int)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2) # 2: thickness

        # get facial features
        embeddings = res['embedding']
        face_embeddings.append(embeddings)

    if sample >= 200:
        break

    # show the frame
    cv2.imshow('frame', frame)
    
    # exit if q is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'): # 1 ms delay, 0xFF: 8 bit binary number
        break
cap.release()
cv2.destroyAllWindows()

In [18]:
len(face_embeddings)

200

#### Optimizing the sample

In [19]:
x_mean = np.asarray(face_embeddings).mean(axis=0)
x_mean.shape

(512,)

In [20]:
# convert into bytes
x_mean_bytes = x_mean.tobytes()

In [21]:
# save key and facial bytes into redis
r.hset(name='academy:register', key=key, value=x_mean_bytes)

1